In [26]:
class Product:
    def __init__(self, id, time, material, cost, profit, efficiency_count=None):
        self.id = id
        self.time = time      # время на единицу (часы)
        self.material = material  # материал на единицу (кг)
        self.cost = cost      # себестоимость единицы (деньги)
        self.profit = profit    # прибыль на единицу (деньги)
        self.quantity = 0  # количество для производства (результат)
        if efficiency_count is None:
            # Расчет эффективности как прибыль на сумму взвешенных ресурсов
            self.efficiency = self.profit / (0.4*self.time + 0.3*self.material + 0.3*self.cost)
        else:
            self.efficiency = efficiency_count(self.time, self.material, self.cost, self.profit)

    def __repr__(self):
        return f"Product {self.efficiency:0.2f}: [{self.id} {self.time} {self.material} {self.cost} {self.profit}]"

    # Функция для сравнения продуктов по эффективности
    def __lt__(self, other):
        return self.efficiency > other.efficiency

# Инициализация данных
products = [
        Product(1, 2.0, 5.0, 100.0, 50.0),   # Продукт 1: 2ч, 5кг, 100д.е., прибыль 50
        Product(2, 3.0, 3.0, 80.0, 40.0),    # Продукт 2: 3ч, 3кг, 80д.е., прибыль 40
        Product(3, 1.5, 7.0, 120.0, 60.0),   # Продукт 3: 1.5ч, 7кг, 120д.е., прибыль 60
        Product(4, 4.0, 2.0, 90.0, 55.0),    # Продукт 4: 4ч, 2кг, 90д.е., прибыль 55
        Product(5, 2.5, 4.0, 110.0, 65.0)    # Продукт 5: 2.5ч, 4кг, 110д.е., прибыль 65
]

# Ограничения ресурсов
max_time = 200.0        # 100 часов
max_material = 200.0    # 200 кг материала
max_budget = 5000.0     # 5000 д.е. бюджета
max_quantity = 20

Жадный алгоритм распределения ресурсов

In [27]:
# Сортируем продукты по эффективности (жадный выбор)
products.sort()
products

[Product 1.88: [4 4.0 2.0 90.0 55.0],
 Product 1.85: [5 2.5 4.0 110.0 65.0],
 Product 1.55: [3 1.5 7.0 120.0 60.0],
 Product 1.55: [1 2.0 5.0 100.0 50.0],
 Product 1.53: [2 3.0 3.0 80.0 40.0]]

Оптимальный план производства

In [29]:
remaining_time = max_time
remaining_material = max_material
remaining_budget = max_budget

for product in products:
    # Максимально возможное количество для текущего продукта
    # с учетом каждого ограничения в отдельности
    max_by_time = remaining_time / product.time
    max_by_material = remaining_material / product.material
    max_by_budget = remaining_budget / product.cost

    # Берем минимум из всех ограничений
    product.quantity = min(max_by_time, max_by_material, max_by_budget)

    # Если нельзя произвести ни одной единицы, переходим к следующему
    if product.quantity <= 0:
        product.quantity = 0
        continue;
    if product.quantity > max_quantity:
        product.quantity = max_quantity

    # Обновляем оставшиеся ресурсы
    remaining_time -= product.quantity * product.time;
    remaining_material -= product.quantity * product.material;
    remaining_budget -= product.quantity * product.cost;

    # Выводим информацию о продукте
    print(f"Продукт {product.id} в количестве {product.quantity:0.2f}")
    print(f'Затраты времени: {product.quantity * product.time:0.2f}')
    print(f'Затраты материала: {product.quantity * product.material:0.2f}')
    print(f'Затраты бюджета: {product.quantity * product.cost:0.2f}')


Продукт 4 в количестве 20.00
Затраты времени: 80.00
Затраты материала: 40.00
Затраты бюджета: 1800.00
Продукт 5 в количестве 20.00
Затраты времени: 50.00
Затраты материала: 80.00
Затраты бюджета: 2200.00
Продукт 3 в количестве 8.33
Затраты времени: 12.50
Затраты материала: 58.33
Затраты бюджета: 1000.00


Расчет итоговых показателей

In [25]:
total_profit = 0
total_time_used = 0
total_material_used = 0
total_cost_used = 0

for product in products:
    total_profit += product.quantity * product.profit
    total_time_used += product.quantity * product.time
    total_material_used += product.quantity * product.material
    total_cost_used += product.quantity * product.cost

print(f"Общая прибыль: {total_profit:0.2f}")
print(f"Использовано времени: {total_time_used:0.2f} / {max_time} \n\t{(total_time_used/max_time)*100:0.2f}%")
print(f"Использовано материала: {total_material_used:0.2f} / {max_material} \n\t{(total_material_used/max_material)*100:0.2f}%")
print(f"Использовано бюджета: {total_cost_used:0.2f} / {max_budget} \n\t{(total_cost_used/max_budget)*100:0.2f}%")

Общая прибыль: 2566.67
Использовано времени: 120.00 / 200.0 
	60.00%
Использовано материала: 200.00 / 200.0 
	100.00%
Использовано бюджета: 4733.33 / 5000.0 
	94.67%
